In [44]:
import pandas as pd
import numpy as np

import yaml

import re

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
                            recall_score, f1_score, log_loss, precision_recall_curve

import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier

import optuna

from typing import Tuple, Set

import warnings
warnings.filterwarnings('ignore')

RAND=42

N_FOLDS = 4

In [2]:
pd.options.mode.chained_assignment = None
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [55]:
def extract_numbers(string):
    return list(map(int, re.findall("'([0-9]+)'", string)))

In [3]:
def extract_words(string):
    return list(map(float, string[1:-1].split()))

In [69]:
def get_metrics(y_test, y_pred, y_score, name = "Default"):
    """Метрики для задачи классификации"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    
    try:
        df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
        df_metrics['Precision'] = precision_score(y_test, y_pred, zero_division=0)
        df_metrics['Recall'] = recall_score(y_test, y_pred, zero_division=0)
        df_metrics['f1'] = f1_score(y_test, y_pred, zero_division=0)
        df_metrics['Logloss'] = log_loss(y_test, y_score)
        
    except ValueError:
        df_metrics['ROC_AUC'] = 0
        df_metrics['Precision'] = 0
        df_metrics['Recall'] = 0
        df_metrics['f1'] = 0
        df_metrics['Logloss'] = 0

    return df_metrics

In [20]:
config_path = "../config/params.yaml"
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config["preprocessing"]
train = config["train"]

In [6]:
config_model = "../config/model_params/lightgbm.yaml"

model_params = yaml.load(open(config_path), Loader=yaml.FullLoader)

# Baseline

In [56]:
df_submission = pd.read_csv(preproc['submission'])
df_submission = df_submission.set_index('index')


df_submission_less_4 = pd.read_csv(preproc['submission_less_4'])
df_submission_less_4 = df_submission_less_4.set_index('index')

In [57]:
df_submission = df_submission['purchases'].apply(extract_numbers)
df_submission_less_4 = df_submission_less_4['purchases'].apply(extract_numbers)

In [7]:
df_train = pd.read_csv(preproc['train_data'])
df_train = df_train.set_index('index')

df_train[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized_tokens,month,reg_code,purchase_size,flag_won,n_unique_okpd2
index,,,,,,,,,,,,
0,3,1,290000.00,2,1,1,[ 0.18815488 0.1963165 0.08348706 0.132998...,2,58.2_2,1,0.0,4
1,2834,1,105000.00,218,1,1,[ 1.88778124e-01 1.99460707e-01 8.44090815e-...,2,62.0_2,1,0.0,4
2,154949,1,98967.50,11235,1,1,[ 0.17555872 0.0838882 0.01939559 0.047119...,2,63.9_2,1,0.0,4
3,147950,0,77460.03,11061,1,1,[ 2.09548737e-01 1.98348963e-01 3.00821184e-...,3,62.0_2,1,0.0,4
4,165990,0,138000.00,11558,1,1,[ 1.55703006e-01 1.47389050e-01 4.36386056e-...,3,62.0_2,1,0.0,4


In [8]:
df_test = pd.read_csv(preproc['test_data'])
df_test = df_test.set_index('index')

df_test[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized_tokens,month,reg_code,purchase_size,flag_won,n_unique_okpd2
index,,,,,,,,,,,,
0,1064,1,181720.00,70,1,1,[ 1.77900028e-01 8.02768195e-02 1.25833983e-...,11,58.2_2,1,0.0,4.0
1,1729,1,167448.00,105,1,1,[ 2.70711475e-01 8.42926477e-02 4.42017172e-...,12,62.0_2,1,0.0,4.0
2,2430,1,200248.16,201,1,1,[ 2.59523023e-01 1.20019309e-01 7.98608701e-...,12,62.0_2,1,0.0,4.0
3,156898,0,190740.00,11266,1,1,[ 0.20131386 0.07292083 -0.01833528 -0.017387...,12,62.0_2,1,0.0,4.0
4,412017,1,2886156.00,9651,100009,0,[ 5.40626203e-02 7.06050537e-02 -3.14055514e-...,11,27.4_77,1,0.0,1.0


In [9]:
df_train['vectorized_tokens'] = df_train['vectorized_tokens'].apply(extract_words)
df_test['vectorized_tokens'] = df_test['vectorized_tokens'].apply(extract_words)

In [10]:
df_train = df_train.astype(preproc['change_type_columns'])
df_test = df_test.astype(preproc['change_type_columns'])

In [11]:
for i in tqdm(range(100)):
    df_train[str(i)] = df_train['vectorized_tokens'].apply(lambda x: x[i])
    df_test[str(i)] = df_test['vectorized_tokens'].apply(lambda x: x[i])

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
df_train[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized_tokens,month,reg_code,purchase_size,...,90,91,92,93,94,95,96,97,98,99
index,,,,,,,,,,,,,,,,,,,,,
0,3,1,290000.00,2,1,1,"[0.18815488, 0.1963165, 0.08348706, 0.13299869...",2,58.2_2,1,...,-0.022450,-0.013303,0.004910,0.041569,-0.018289,-0.039090,-0.042309,-0.085248,0.049973,-0.010035
1,2834,1,105000.00,218,1,1,"[0.188778124, 0.199460707, 0.0844090815, 0.123...",2,62.0_2,1,...,-0.036159,0.006485,0.025578,-0.017571,0.019805,-0.007594,0.014430,0.002117,-0.015829,0.030955
2,154949,1,98967.50,11235,1,1,"[0.17555872, 0.0838882, 0.01939559, 0.04711946...",2,63.9_2,1,...,-0.006030,0.043595,-0.059633,-0.031424,-0.001296,-0.030366,0.040771,0.122105,-0.059927,-0.049989
3,147950,0,77460.03,11061,1,1,"[0.209548737, 0.198348963, 0.0300821184, 0.104...",3,62.0_2,1,...,0.003393,0.004516,0.006793,-0.077847,0.016764,0.002300,0.040950,0.075495,-0.047596,0.044951
4,165990,0,138000.00,11558,1,1,"[0.155703006, 0.14738905, 0.0436386056, 0.0713...",3,62.0_2,1,...,-0.004449,-0.006018,0.016005,0.008470,-0.031247,-0.014706,-0.051110,-0.057549,0.066321,-0.014859


In [19]:
df_test[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized_tokens,month,reg_code,purchase_size,...,90,91,92,93,94,95,96,97,98,99
index,,,,,,,,,,,,,,,,,,,,,
0,1064,1,181720.00,70,1,1,"[0.177900028, 0.0802768195, 0.125833983, 0.188...",11,58.2_2,1,...,-0.001060,0.015380,-0.020307,-0.076040,0.040264,-0.010360,0.070702,0.069413,-0.050787,0.006304
1,1729,1,167448.00,105,1,1,"[0.270711475, 0.0842926477, 0.0442017172, 0.07...",12,62.0_2,1,...,0.037985,0.035489,-0.015627,-0.010494,0.001370,0.004961,-0.019968,0.067377,-0.009527,-0.014030
2,2430,1,200248.16,201,1,1,"[0.259523023, 0.120019309, 0.0798608701, 0.101...",12,62.0_2,1,...,0.027756,0.027557,-0.007591,-0.033967,0.034716,-0.003503,-0.000300,0.053355,0.003905,0.012280
3,156898,0,190740.00,11266,1,1,"[0.20131386, 0.07292083, -0.01833528, -0.01738...",12,62.0_2,1,...,0.049544,0.045962,-0.016287,-0.028851,0.029081,0.076335,0.036749,0.148115,-0.042723,-0.050340
4,412017,1,2886156.00,9651,100009,0,"[0.0540626203, 0.0706050537, -0.0314055514, -0...",11,27.4_77,1,...,0.090559,0.064498,0.013654,0.058032,0.072006,-0.095470,-0.099567,-0.021023,-0.021823,0.113911


In [77]:
def find_closest_vectors(target_vectors: np.ndarray, vector_space: np.ndarray, 
                         k: int =5) -> np.ndarray:
    """
    Find the k vectors in the given vector_space that are closest to the given target_vectors.

    Args:
        target_vectors (numpy.ndarray): A 2D numpy array of shape (n, m) 
            where n is the number of target vectors and m is the dimensionality of the vectors.
        vector_space (numpy.ndarray): A 2D numpy array of shape (p, m) 
            where p is the number of vectors in the vector space and m is the 
            dimensionality of the vectors.
        k (int): The number of closest vectors to return. Default is 5.

    Returns:
        numpy.ndarray: A 1D numpy array of length k containing the indices of 
        the closest vectors in the vector space.
    """
    sum_distances = []
    
    
    mms = MinMaxScaler()
    
    
    #Нормализуем векторы
    target_vectors = mms.fit_transform(target_vectors)
    vector_space = mms.transform(vector_space)
    
    
    for vector in vector_space:
        # создаем массив расстояний между вектором в пространстве и целевыми векторами
        distances = np.linalg.norm(target_vectors - vector, axis=1)
        # сохраняем сумму расстояний между вектором в пространстве и целевыми векторами
        sum_distances.append(np.sum(distances))
    
    
    # выбираем набор векторов с минимальной суммой расстояний до каждого вектора в target_vectors
    if k >= vector_space.shape[0]:
        return np.nonzero(vector_space)
    
    
    return np.argpartition(sum_distances, k)[:k]

In [63]:
def baseline_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, params = None) -> pd.DataFrame:
    """
    Trains a LightGBM classifier on the training set and returns the score on the test set.

    Args:
        df_tr (pandas.DataFrame): A pandas DataFrame containing the training set.
        df_t (pandas.DataFrame): A pandas DataFrame containing the test set.

    Returns:
        pandas.DataFrame: The score on the test set.

    """

    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
    

    model = LGBMClassifier(random_state=train['random_state'], class_weight='balanced', n_jobs=-1)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    y_score = model.predict_proba(x_test)

    return get_metrics(y_test, y_pred, y_score)

In [13]:
def filter_data(df_train, df_sup, sup):
    unique_reg_okpd = df_train[df_train['supplier'] == sup]['reg_code'].unique()
    
    # фильтруем train и test на основе уникальных reg_code поставщиков 
    df_sup_train = df_train[df_train['reg_code'].isin(unique_reg_okpd)]
    df_sup_test = df_test[df_test['reg_code'].isin(unique_reg_okpd)]
    
    if df_sup_test.empty:
        df_sup_test = df_test
        
    return df_sup_train, df_sup_test    

In [88]:
recall_less_4 = []

for sup in tqdm(df_submission_less_4.index[:500]):
    
    df_sup_train, df_sup_test = filter_data(df_train, df_test, sup)
    
    vectors = df_sup_train[df_train['supplier'] == sup]
    
    
    # удаляем выделенные столбцы и дубликаты
    vectors = vectors.drop(columns=train['drop_columns_lgbm']).drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=train['drop_columns_lgbm']).drop_duplicates()
    
    vectors = vectors.set_index('purchase')
    df_sup_test = df_sup_test.set_index('purchase')
    
    
    idx = find_closest_vectors(vectors, df_sup_test)
    y_pred = df_sup_test.iloc[idx].index
    
    recall = len(y_pred & df_submission_less_4[sup]) \
                 / len(df_submission_less_4[sup])

    recall_less_4.append(recall)
    
np.mean(recall_less_4)

  0%|          | 0/16 [00:00<?, ?it/s]

0.640625

In [74]:
metrics = pd.DataFrame()

for sup in tqdm(df_submission.index[:500]):
    
    # фильтруем датасеты
    df_sup_train, df_sup_test = filter_data(df_train, df_test, sup)

    
    # удаляем выделенные столбцы и дубликаты
    df_sup_train = df_sup_train.drop(columns=train['drop_columns_lgbm']).drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=train['drop_columns_lgbm']).drop_duplicates()

    df_sup_test = df_sup_test.set_index('purchase')
    df_sup_train = df_sup_train.set_index('purchase')


    # добавляем метки для обучения алгоритма классификации 
    df_sup_train['target'] = df_sup_train.index.isin(df_submission[sup]).astype(int)
    df_sup_test['target'] = df_sup_test.index.isin(df_train[df_train['supplier'] == sup]['purchase']
                                                   .unique()).astype(int)

    metrics = pd.concat([metrics, baseline_lgbm(df_sup_train, df_sup_test)])

metrics.mean(axis=0)

  0%|          | 0/12 [00:00<?, ?it/s]

Accuracy     1.0
ROC_AUC      0.0
Precision    0.0
Recall       0.0
f1           0.0
Logloss      0.0
dtype: float64

# Tune

In [28]:
def objective(trial: optuna.Trial, x: pd.DataFrame, y: pd.Series, learning_rate=None) -> float:
    """
    This function defines the objective function for an Optuna study to tune hyperparameters
    for a LightGBM binary classification model. 

    Args:
        trial (optuna.Trial): A trial corresponding to a set of hyperparameters.
        x (pd.DataFrame): The features to be used for training and validation.
        y (pd.Series): The target variable for training and validation.

    Returns:
        float: The mean of the cross-validation AUC-ROC scores for the given set of hyperparameters.
    """
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [400]),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'learning_rate': trial.suggest_categorical('learning_rate', [learning_rate]),
#         'max_bin': trial.suggest_int('max_bin', 200, 800),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 150),
#         'max_depth': trial.suggest_int('max_depth', 4, 12),
#         'min_child_samples': trial.suggest_int('min_child_samples', 200, 20000, step=200),
#         'lambda_l1': trial.suggest_int('lambda_l1', 0, 50),
#         'lambda_l2': trial.suggest_int('lambda_l2', 0, 50),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0.001, 0.1),
#         'objective': 'binary',
#         'metric': 'auc',
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.3, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.3, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 2, 6),
#         'random_state': train['random_state'],
    }

    cv_pred = np.empty(train['N_FOLDS'])
    cv = StratifiedKFold(n_splits=train['N_FOLDS'], shuffle=True, random_state=train['random_state'])

    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):
        x_train_, x_val_ = x.iloc[train_idx], x.iloc[test_idx]
        y_train_, y_val_ = y.iloc[train_idx], y.iloc[test_idx]

        pruning = optuna.integration.LightGBMPruningCallback(trial, 'auc')

        model = LGBMClassifier(
            class_weight='balanced',
            n_jobs=-1,
            **params
        )
        model.fit(x_train_, y_train_,
                  eval_metric='auc',
                  eval_set=[(x_val_, y_val_)],
                  early_stopping_rounds=100,
                  callbacks=[pruning],
                  verbose=0)

        y_pred = model.predict(x_val_)
        y_proba = model.predict_proba(x_val_)[:, 1]

        cv_pred[fold] = roc_auc_score(y_val_, y_proba)
    return (np.mean(cv_pred))

In [33]:
def tune_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, learning_rate=None) -> pd.DataFrame:
    
    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
     
        
    func = lambda trial: objective(trial, x_train, y_train, learning_rate)

    study = optuna.create_study(direction="maximize")
    study.optimize(func, n_trials=50, n_jobs=-1)
    
    return study.best_params

In [32]:
def open_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
def save_file(file_path, data):       
    with open(file_path, 'w') as file:
        yaml.dump(data, file)

In [31]:
def tune_algorithm(df_train: pd.DataFrame, df_test: pd.DataFrame, 
                   df_submission: pd.DataFrame, df: pd.DataFrame, sup, learning_rate=None):
    
    # фильтруем датасеты
    df_sup_train, df_sup_test = filter_data(df_train, df_test, sup)

    
    # удаляем выделенные столбцы и дубликаты
    df_sup_train = df_sup_train.drop(columns=train['drop_columns_lgbm']).drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=train['drop_columns_lgbm']).drop_duplicates()

    df_sup_test = df_sup_test.set_index('purchase')
    df_sup_train = df_sup_train.set_index('purchase')


    # добавляем метки для обучения алгоритма классификации 
    df_sup_train['target'] = df_sup_train.index.isin(df_submission[sup]).astype(int)
    df_sup_test['target'] = df_sup_test.index.isin(df_train[df_train['supplier'] == sup]['purchase']
                                                   .unique()).astype(int)

    params = tune_lgbm(df_sup_train, df_sup_test, sup, learning_rate)
        
    return params

In [34]:
data = open_file(file_path)

for sup in tqdm(df_submission.index[:500]):

    if sup in data:
        params = tune_algorithm(df_train, df_test, df_submission, df, sup, **data[sup])
        data[sup].extend(params)
        
    # Если ключа еще нет, создать новый список
    else:
        params = tune_algorithm(df_train, df_test, df_submission, df, sup)        
        data[sup] = params

save_file(file_path, data)

  0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=8, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8
[LightGBM] [Warning] bagging_fraction is set=0.6751165316026022, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6751165316026022
[LightGBM] [Warning] feature_fraction is set=0.4683249767096229, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4683249767096229
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] lambda_l1 is set=6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6
[LightGBM] [Warning] bagging_fraction is set=0.5035541546690397, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5035541546690397
[LightGBM] [Warning] feature_fraction is set=0.535969077444137, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.53596907

[LightGBM] [Warning] lambda_l1 is set=36, reg_alpha=0.0 will be ignored. Current value: lambda_l1=36
[LightGBM] [Warning] bagging_fraction is set=0.6920323854263122, subsample=1.0 will be ignored. Current value: bagging_fraction

limit_output extension: Maximum message size of 10000 exceeded with 14658 characters

In [29]:
def classifier_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, params = None) -> pd.DataFrame:
    """
    Trains a LightGBM classifier on the training set and returns the score on the test set.

    Args:
        df_tr (pandas.DataFrame): A pandas DataFrame containing the training set.
        df_t (pandas.DataFrame): A pandas DataFrame containing the test set.

    Returns:
        pandas.DataFrame: The score on the test set.

    """

    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
    

    model = LGBMClassifier(class_weight='balanced', n_jobs=-1, **params)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    y_score = model.predict_proba(x_test)

    return get_metrics(y_test, y_pred, y_score)

In [33]:
data = open_file(file_path)

In [64]:
metrics = pd.DataFrame()

for sup in tqdm(df_submission.index[:500]):

    # фильтруем датасеты
    df_sup_train, df_sup_test = filter_data(df_train, df_test, sup)

    
    # удаляем выделенные столбцы и дубликаты
    df_sup_train = df_sup_train.drop(columns=train['drop_columns_lgbm']).drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=train['drop_columns_lgbm']).drop_duplicates()

    df_sup_test = df_sup_test.set_index('purchase')
    df_sup_train = df_sup_train.set_index('purchase')


    # добавляем метки для обучения алгоритма классификации 
    df_sup_train['target'] = df_sup_train.index.isin(df_submission[sup]).astype(int)
    df_sup_test['target'] = df_sup_test.index.isin(df_train[df_train['supplier'] == sup]['purchase']
                                                   .unique()).astype(int)
    
    params = data[sup]

    metrics = pd.concat([metrics, classifier_lgbm(df_sup_train, df_sup_test, params=params)])

  0%|          | 0/12 [00:00<?, ?it/s]

NameError: name 'edit_data' is not defined

In [36]:
metrics.mean(axis=0)

Accuracy     0.730974
ROC_AUC      0.743162
Precision    0.090660
Recall       0.638708
f1           0.139746
Logloss      0.436611
dtype: float64